In [3]:
# ЗАГОТОВКА КОДА ДЛЯ УЧАЩИХСЯ


#с проверкой возможности действия
import numpy as np
from random import randint
import pygame
pygame.init()


class Environment:
    # Settings
    color1 = (228, 206, 242)  # фон
    color2 = (94, 53, 242)  # (161, 115, 191)  # стенки
    color3 = (29, 30, 38)  # линии
    color4 = (242, 5, 135)  # финиш и старт

    def __init__(self, environment_rows, environment_columns, screen, scale):
        self.environment_rows = environment_rows
        self.environment_columns = environment_columns
        self.finish_reward = 1000000
        self.reward = -1
#         self.dangerous_reward = -100  
        self.rewards = np.full((self.environment_rows, self.environment_columns), self.reward)
        self.maze = np.ones((self.environment_rows, self.environment_columns), dtype=np.uint8)
        self.screen = screen
        self.scale = scale

    def generate_maze(self):
        aisles = {}
        aisles[1] = [1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13]
        aisles[2] = [1, 3, 5, 7, 9, 13]
        aisles[3] = [1, 3, 4, 5, 7, 9, 11, 12, 13]
        aisles[4] = [1, 7, 9, 11]
        aisles[5] = [1, 2, 3, 4, 5, 7, 9, 11, 12, 13]
        aisles[6] = [5, 7, 11, 13]
        aisles[7] = [1, 2, 3, 5, 7, 8, 9, 11, 13]
        aisles[8] = [1, 3, 5, 9, 11, 13]
        aisles[9] = [1, 3, 4, 5, 7, 9, 10, 11, 13]
        aisles[10] = [1, 7, 11]
        aisles[11] = [i for i in range(1, self.environment_columns-1)]

        self.generate_rewards(aisles)  # Generate rewards

        # создаем проходы в лабиринте (0 - проход)
        for row_index in range(1, self.environment_rows-1):
            for column_index in aisles[row_index]:
                self.maze[row_index, column_index] = 0
        self.maze[1, 0] = 2  # старт
        self.maze[11, 14] = 3  # финиш
        """self.maze[3, 14] = 4  # ловушка
        self.maze[8, 9] = 4  # ловушка
        self.maze[8, 14] = 4  # ловушка"""

    def generate_rewards(self, aisles):
        # В проходах награды будут равны ???      
        for row_index in range(1, self.environment_rows-1):
            for column_index in aisles[row_index]:
                self.rewards[row_index, column_index] = self.reward
        self.rewards[11, 14] = self.finish_reward
        '''Можно добавить ловушки по желанию'''
        #self.rewards[3, 14] = self.dangerous_reward
        #self.rewards[8, 9] = self.dangerous_reward
        #self.rewards[8, 14] = self.dangerous_reward

    def draw_maze(self):
        for row in range(0, self.environment_rows):  # Перебираем строки в матрице
            for column in range(0, self.environment_columns):  # Перебираем столбцы в матрице
                # Draw walls
                if self.maze[row, column] == 1:
                    pygame.draw.rect(self.screen, self.color2, ((column + 1) * self.scale, \
                                                                (row + 1) * self.scale, self.scale, self.scale))
                # Draw start and finish
                elif self.maze[row, column] == 2 or self.maze[row, column] == 3:
                    pygame.draw.rect(self.screen, self.color4, ((column + 1) * self.scale, \
                                                                (row + 1) * self.scale, self.scale, self.scale))
                # Draw dangerous cells
                elif self.maze[row, column] == 4:
                    pygame.draw.rect(self.screen, self.color5, ((column + 1) * self.scale, \
                                                                (row + 1) * self.scale, self.scale, self.scale))

    def draw_lines(self):
        for i in range(1, self.environment_rows):
            # горизонтальные
            pygame.draw.line(self.screen, self.color3, [self.scale, (i + 1) * self.scale], \
                             [(self.environment_columns + 1) * self.scale, (i + 1) * self.scale])
        # рисуем жирно первую и последнюю линии
        th = 3  # толщина линий
        pygame.draw.line(self.screen, self.color3, [self.scale, self.scale], \
                         [(self.environment_columns + 1) * self.scale, self.scale], th)
        pygame.draw.line(self.screen, self.color3, [self.scale, (self.environment_rows + 1) * self.scale], \
                         [(self.environment_columns + 1) * self.scale, (self.environment_rows + 1) * self.scale], th)

        for i in range(1, self.environment_columns):
            # вертикальные
            pygame.draw.line(self.screen, self.color3, [(i + 1) * self.scale, self.scale], \
                             [(i + 1) * self.scale, (self.environment_rows + 1) * self.scale])
        # рисуем жирно первую и последнюю линии
        th = 3  # толщина линий
        pygame.draw.line(self.screen, self.color3, [self.scale, self.scale], \
                         [self.scale, (self.environment_rows + 1) * self.scale], th)
        pygame.draw.line(self.screen, self.color3, [(self.environment_columns + 1) * self.scale, self.scale], \
                         [(self.environment_columns + 1) * self.scale, (self.environment_rows + 1) * self.scale], th)

    def draw_row_index(self):
        # рисуем индексы строк
        row_index = 0
        row_index_x = self.scale // 2
        row_index_y = self.scale + self.scale // 2
        for i in range(self.environment_rows):
            index = pygame.font.SysFont('verdana', 15).render(str(row_index), True, (0, 0, 0))
            self.screen.blit(index, (row_index_x, row_index_y))
            row_index_y += self.scale
            row_index += 1

    def draw_column_index(self):
        # рисуем индексы столбцов
        column_index = 0
        column_index_x = self.scale + self.scale // 2
        column_index_y = self.scale // 2
        for i in range(self.environment_columns):
            index = pygame.font.SysFont('verdana', 15).render(str(column_index), True, (0, 0, 0))
            self.screen.blit(index, (column_index_x, column_index_y))
            column_index_x += self.scale
            column_index += 1

    def draw_environment(self):
        self.draw_maze()
        self.draw_lines()
        self.draw_row_index()
        self.draw_column_index()


# _________________________________________________________________________


class Agent(Environment):
    # Settings
    start_row_index = 1  # Начальное положение агента
    start_column_index = 0

    def __init__(self):
        super().__init__(environment_rows, environment_columns, screen, scale)
        self.image = pygame.transform.scale(pygame.image.load('data/sonic.png'), (scale // 2, scale))
        self.current_row_index = self.start_row_index
        self.current_column_index = self.start_column_index
        self.actions = ['up', 'right', 'down', 'left']

    def draw_agent(self):
        screen.blit(self.image, (self.current_column_index * self.scale + self.scale, \
                                 self.current_row_index * self.scale + self.scale))


# _________________________________________________________________________


#создадим класс для обучения
class Q_learning(Agent):
    
    # Settings
    epsilon = 0.2
    gamma = 0.98
    alpha = 0.9
    
    def __init__(self, environment_rows, environment_columns, screen, scale): 
        super().__init__()
        # Генерируем q-таблицу как 3-мерный массив
        self.q_table = np.zeros((environment_rows, environment_columns, 4))
        self.episod_number = 1
        self.finish = 0
        
    def inf_about_episods(self):
        inf = pygame.font.SysFont('verdana', 15).render('Эпизод № ' + str(self.episod_number), True, (0,0,0))
        screen.blit(inf, (700, 100))
        
    # Здесь нужно описать методы, необходимые для обучения методом q-learning
    def get_next_possible_states(self):
        states = []
        # Здесь должен быть код
        if self.current_row_index > 0 and self.maze[self.current_row_index - 1, self.current_column_index] != 1:
            states.append([self.current_row_index - 1, self.current_column_index, 0])
        
        if self.current_column_index < self.environment_columns - 1 and self.maze[self.current_row_index, self.current_column_index + 1] != 1:
            states.append([self.current_row_index, self.current_column_index + 1, 1])

        if self.current_row_index < self.environment_rows - 1 and self.maze[self.current_row_index + 1, self.current_column_index] != 1:
            states.append([self.current_row_index + 1, self.current_column_index, 2])

        if self.current_column_index > 0 and self.maze[self.current_row_index, self.current_column_index - 1] != 1:
            states.append([self.current_row_index, self.current_column_index - 1, 3])
            
        return states

    def is_finish(self):
        if self.maze[self.current_row_index, self.current_column_index] == 3:
            self.finish += 1
            return True
        else:
            return False
    
    def get_values_of_next_possible_states(self, states_):
        values_of_states = []
        for state in states_:
            value = max(self.q_table[state[0]][state[1]])
            values_of_states.append(value)
        return values_of_states

    # определим эпсилон жадный алгоритм, который будет выбирать следующее состояние
    def get_next_state(self):
        # получаем список возможных следующих состояний
        next_states = self.get_next_possible_states()
        # получаем ценности возможных следующих состояний
        next_states_lol = []
        # если сулчайно выбранное значение от 0 до 1 меньше, чем эпсилон
        # тогда выбираем наиболее ценное состояние из таблицы ценностей
        for i in range(len(next_states)):
            next_states_lol.append(self.q_table[self.current_row_index, self.current_column_index, next_states[i][2]])
        if np.random.random() < self.epsilon:
            index_of_next_state = np.argmax(next_states_lol)
        else:  # выбираем случайное состояние
            index_of_next_state = randint(0, len(next_states) - 1)
        next_state_ = next_states[index_of_next_state]
        return next_state_

    def update_table_of_values(self):
        new_state = self.get_next_state()
        last_reward = self.rewards[new_state[0], new_state[1]]
        self.q_table[self.current_row_index, self.current_column_index, new_state[2]] = self.q_table[self.current_row_index, self.current_column_index, new_state[2]] + self.alpha * (last_reward + self.gamma * max(self.q_table[new_state[0], new_state[1]]) - self.q_table[self.current_row_index, self.current_column_index, new_state[2]])
        self.current_row_index = new_state[0]
        self.current_column_index = new_state[1]

    # Метод learning будет отвечать за обучение агента
    def learning(self):
        if self.is_finish():
            self.current_row_index = 1
            self.current_column_index = 0
        else:
            self.update_table_of_values()
        if self.epsilon != 1:
            if self.finish > 10:
                self.epsilon = 0.3
            if self.finish > 20:
                self.epsilon = 0.4
            if self.finish > 30:
                self.epsilon = 0.5
            if self.finish > 40:
                self.epsilon = 0.6
            if self.finish > 50:
                self.epsilon = 0.7
            if self.finish > 75:
                self.epsilon = 0.8
            if self.finish > 100:
                self.epsilon = 0.9

        # По желанию можно придумать, как отрисовывать ценности действий в среде
    def draw_values(self):
        for column in range(0, self.environment_columns):  # перебираем строки в матрице
            for row in range(0, self.environment_rows):  # перебираем столбцы в матрице
                if self.maze[row, column] != 1:
                    value = round(self.q_table[row, column, 0], 2)
                    print_value = pygame.font.SysFont('verdana', 10).render(str(value), True, (0, 0, 0))
                    self.screen.blit(print_value, (column * self.scale + self.scale + 10, row * self.scale + self.scale))
                    value = round(self.q_table[row, column, 1], 2)
                    print_value = pygame.font.SysFont('verdana', 10).render(str(value), True, (0, 0, 0))
                    self.screen.blit(print_value, (column * self.scale + self.scale + 20, row * self.scale + self.scale + 15))
                    value = round(self.q_table[row, column, 2], 2)
                    print_value = pygame.font.SysFont('verdana', 10).render(str(value), True, (0, 0, 0))
                    self.screen.blit(print_value, (column * self.scale + self.scale + 10, row * self.scale + self.scale + 27))
                    value = round(self.q_table[row, column, 3], 2)
                    print_value = pygame.font.SysFont('verdana', 10).render(str(value), True, (0, 0, 0))
                    self.screen.blit(print_value, (column * self.scale + self.scale, row * self.scale + self.scale + 15))

    def inf_about_episods(self):
        inf = pygame.font.SysFont('verdana', 15).render('Эпизод № ' + str(self.finish), True, (0, 0, 0))
        screen.blit(inf, (800, 80))



environment_rows = 13
environment_columns = 15


FPS = 10000
clock = pygame.time.Clock()
color1 = (228, 206, 242)  # фон
color2 = (94, 53, 242)  # (161, 115, 191)  # стенки
color3 = (29, 30, 38)  # линии
color4 = (242, 5, 135)  # финиш и старт
color5 =  (140, 92, 242) # ловушки
background_color = color1
scale = 40
(width, height) = (environment_columns * scale + 350 + scale, environment_rows * scale + 50 + scale)
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption('Training')         #заголовок окна
screen.fill(background_color)
pygame.display.flip()



#создаем персонажа игры как объекта класса Q_learning
agent_sonic = Q_learning(environment_rows, environment_columns, screen, scale)
agent_sonic.generate_maze()

is_last = 0
# while agent_sonic.finish <= 40:
#     agent_sonic.learning()
#     if agent_sonic.finish != is_last:
#         is_last = agent_sonic.finish
#         print(is_last)
# agent_sonic.epsilon = 1

running = True
while running == True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
    screen.fill(background_color)
    #if agent_sonic.episod_number > 100:
    agent_sonic.draw_environment()
    agent_sonic.learning()
    agent_sonic.draw_agent()
#     agent_sonic.draw_values()
    agent_sonic.inf_about_episods()
    #agent_sonic.draw_values()
    
    clock.tick(FPS)
    pygame.display.flip()

matrix = agent_sonic.q_table

for i in range(len(matrix)):
    for j in range(len(matrix[i])):
        for k in range(4):
            matrix[i][j][k] = round(matrix[i][j][k], 2)
        
matrix = np.array(matrix, dtype=np.unicode_)

with open("table.txt", "w", encoding='utf-8') as f:
    f.write(str(agent_sonic.finish) + '\n')
    for i in matrix:
        for j in i:
            f.write(" ".join(j) + '\n')

pygame.quit()

In [ ]:

# СЧИТЫВАНИЕ ЦЕННОСТЕЙ С ФАЙЛА
import numpy as np
import pygame
pygame.init()


class Environment:
    # Settings
    color1 = (228, 206, 242)  # фон
    color2 = (94, 53, 242)  # (161, 115, 191)  # стенки
    color3 = (29, 30, 38)  # линии
    color4 = (242, 5, 135)  # финиш и старт
    color5 =  (140, 92, 242) # ловушки

    def __init__(self, environment_rows, environment_columns, screen, scale):
        self.environment_rows = environment_rows
        self.environment_columns = environment_columns
        self.finish_reward = 1000.
        self.reward = -1.
        self.dangerous_reward = -100.
        self.rewards = np.full((self.environment_rows, self.environment_columns), 0)
        self.maze = np.ones((self.environment_rows, self.environment_columns), dtype=np.uint8)
        self.screen = screen
        self.scale = scale

    def generate_maze(self):
        # создаем словарь для проходов
        aisles = {}
        aisles[1] = [1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13]
        aisles[2] = [1, 3, 5, 7, 9, 13]
        aisles[3] = [1, 3, 4, 5, 7, 9, 11, 12, 13]
        aisles[4] = [1, 7, 9, 11]
        aisles[5] = [1, 2, 3, 4, 5, 7, 9, 11, 12, 13]
        aisles[6] = [5, 7, 11, 13]
        aisles[7] = [1, 2, 3, 5, 7, 8, 9, 11, 13]
        aisles[8] = [1, 3, 5, 9, 11, 13]
        aisles[9] = [1, 3, 4, 5, 7, 9, 10, 11, 13]
        aisles[10] = [1, 7, 11]
        aisles[11] = [i for i in range(1, self.environment_columns-1)]

        self.generate_rewards(aisles)  # Generate rewards

        # создаем проходы в лабиринте (0 - проход)
        for row_index in range(1, self.environment_rows-1):
            for column_index in aisles[row_index]:
                self.maze[row_index, column_index] = 0
        self.maze[1, 0] = 2  # старт
        self.maze[11, 14] = 3  # финиш
        """self.maze[3, 14] = 4  # ловушка
        self.maze[8, 9] = 4  # ловушка
        self.maze[8, 14] = 4  # ловушка"""

    def generate_rewards(self, aisles):
        # В проходах награды будут равны -1        
        for row_index in range(1, self.environment_rows-1):
            for column_index in aisles[row_index]:
                self.rewards[row_index, column_index] = self.reward
        self.rewards[11, 14] = self.finish_reward
        """self.rewards[3, 14] = self.dangerous_reward
        self.rewards[8, 9] = self.dangerous_reward
        self.rewards[8, 14] = self.dangerous_reward"""

    def draw_maze(self):
        for row in range(0, self.environment_rows):  # Перебираем строки в матрице
            for column in range(0, self.environment_columns):  # Перебираем столбцы в матрице
                # Draw walls
                if self.maze[row, column] == 1:
                    pygame.draw.rect(self.screen, self.color2, ((column + 1) * self.scale, \
                                                                (row + 1) * self.scale, self.scale, self.scale))
                # Draw start and finish
                elif self.maze[row, column] == 2 or self.maze[row, column] == 3:
                    pygame.draw.rect(self.screen, self.color4, ((column + 1) * self.scale, \
                                                                (row + 1) * self.scale, self.scale, self.scale))
                # Draw dangerous cells
                elif self.maze[row, column] == 4:
                    pygame.draw.rect(self.screen, self.color5, ((column + 1) * self.scale, \
                                                                (row + 1) * self.scale, self.scale, self.scale))

    def draw_lines(self):
        for i in range(1, self.environment_rows):
            # горизонтальные
            pygame.draw.line(self.screen, self.color3, [self.scale, (i + 1) * self.scale], \
                             [(self.environment_columns + 1) * self.scale, (i + 1) * self.scale])
        # рисуем жирно первую и последнюю линии
        th = 3  # толщина линий
        pygame.draw.line(self.screen, self.color3, [self.scale, self.scale], \
                         [(self.environment_columns + 1) * self.scale, self.scale], th)
        pygame.draw.line(self.screen, self.color3, [self.scale, (self.environment_rows + 1) * self.scale], \
                         [(self.environment_columns + 1) * self.scale, (self.environment_rows + 1) * self.scale], th)

        for i in range(1, self.environment_columns):
            # вертикальные
            pygame.draw.line(self.screen, self.color3, [(i + 1) * self.scale, self.scale], \
                             [(i + 1) * self.scale, (self.environment_rows + 1) * self.scale])
        # рисуем жирно первую и последнюю линии
        th = 3  # толщина линий
        pygame.draw.line(self.screen, self.color3, [self.scale, self.scale], \
                         [self.scale, (self.environment_rows + 1) * self.scale], th)
        pygame.draw.line(self.screen, self.color3, [(self.environment_columns + 1) * self.scale, self.scale], \
                         [(self.environment_columns + 1) * self.scale, (self.environment_rows + 1) * self.scale], th)

    def draw_row_index(self):
        # рисуем индексы строк
        row_index = 0
        row_index_x = self.scale // 2
        row_index_y = self.scale + self.scale // 2
        for i in range(self.environment_rows):
            index = pygame.font.SysFont('verdana', 15).render(str(row_index), True, (0, 0, 0))
            self.screen.blit(index, (row_index_x, row_index_y))
            row_index_y += self.scale
            row_index += 1

    def draw_column_index(self):
        # рисуем индексы столбцов
        column_index = 0
        column_index_x = self.scale + self.scale // 2
        column_index_y = self.scale // 2
        for i in range(self.environment_columns):
            index = pygame.font.SysFont('verdana', 15).render(str(column_index), True, (0, 0, 0))
            self.screen.blit(index, (column_index_x, column_index_y))
            column_index_x += self.scale
            column_index += 1

    def draw_environment(self):
        self.draw_maze()
        self.draw_lines()
        self.draw_row_index()
        self.draw_column_index()


# _________________________________________________________________________


class Agent(Environment):
    # Settings
    start_row_index = 1  # Начальное положение агента
    start_column_index = 0

    def __init__(self):
        super().__init__(environment_rows, environment_columns, screen, scale)
        self.image = pygame.transform.scale(pygame.image.load('data/sonic.png'), (scale // 2, scale))
        self.current_row_index = self.start_row_index
        self.current_column_index = self.start_column_index
        self.actions = ['up', 'right', 'down', 'left']

    def draw_agent(self):
        screen.blit(self.image, (self.current_column_index * self.scale + self.scale, \
                                 self.current_row_index * self.scale + self.scale))


# _________________________________________________________________________


#создадим класс для обучения
class Q_learning(Agent):
    
    # Settings
    epsilon = 1.
    gamma = 0.99
    alpha = 0.02  # learning rate
    
    def __init__(self, environment_rows, environment_columns, screen, scale, learned = False): 
        super().__init__()
        self.episod_number = 1
        self.q_table = np.zeros((environment_rows, environment_columns, 4)) if not learned else self.read_file()
        
    
    def read_file(self):
        file = open('table.txt', 'r')
        data = file.readlines()
        self.episod_number = int(data[0])
        data.pop(0)
        q_table = np.zeros((self.environment_rows, self.environment_columns, 4))
        str_ind = 0
        for row_index in range(self.environment_rows):
            for column_index in range(self.environment_columns):
                for i, value in enumerate(data[str_ind].split(' ')):
                    if value != "\n":
                        q_table[row_index][column_index][i] = float(value)
                str_ind += 1

        """for index, value in enumerate(data):
            for idex_row in range(environment_rows):
                for i, weight in enumerate(value.split(' ')):
                    if(weight == "\n"):
                        continue
                    table_of_values[index//15][idex_row][i] = float(weight )"""
        return q_table
    
    
    #зададим метод, определяющий является ли положение агента конечным
    def is_terminal_state(self):
      #если награда в этом положение равна -1, то это не конечное положение (дорожка)
        if self.rewards[self.current_row_index, self.current_column_index] == self.finish_reward:
            return True
        else:
            return False


    #определим эпсилон жадный алгоритм, который будет выбирать следующее действие
    def get_next_action(self):
        #если сулчайно выбранное значение от 0 до 1 меньше, чем эпсилон
        #тогда выбираем наиболее ценное действие из q-таблицы для этого состояния
        if np.random.random() < self.epsilon:
            action =  np.argmax(self.q_table[self.current_row_index, self.current_column_index]) 
        else: #choose a random action
            action = np.random.randint(4)        
        return action
    
    def check_action(self, action_index):
        if self.actions[action_index] == 'up' and self.current_row_index > 0 and \
            self.maze[self.current_row_index-1, self.current_column_index] == 1:
            return False
        elif self.actions[action_index] == 'right' and self.current_column_index < environment_columns - 1 and \
            self.maze[self.current_row_index, self.current_column_index+1] == 1:
            return False
        elif self.actions[action_index] == 'down' and self.current_row_index < environment_rows - 2 and \
        self.maze[self.current_row_index+1, self.current_column_index] == 1:
            return False
        elif self.actions[action_index] == 'left' and self.current_column_index > 0 and \
        self.maze[self.current_row_index, self.current_column_index-1] == 1:
            return False
        else:
            return True
    
    #зададим метод, определяющий следующее положение, используя выбранное действие
    def get_next_location(self, action_index):
        if not self.is_terminal_state():
            new_row_index = self.current_row_index
            new_column_index = self.current_column_index
            if self.actions[action_index] == 'up' and self.current_row_index > 0:
                new_row_index -= 1

            elif self.actions[action_index] == 'right' and self.current_column_index < environment_columns - 1:
                new_column_index += 1

            elif self.actions[action_index] == 'down' and self.current_row_index < environment_rows - 2:
                new_row_index += 1

            elif self.actions[action_index] == 'left' and self.current_column_index > 0:
                new_column_index -= 1
        else:
            new_row_index, new_column_index = self.start_row_index, self.start_column_index
            #self.episod_number += 1
            
        return new_row_index, new_column_index
        
    
    def update_q_table(self, old_row_index, old_column_index, action_index):
        
        reward = self.rewards[self.current_row_index, self.current_column_index]
        old_q_value = self.q_table[old_row_index, old_column_index, action_index]
        temporal_difference = reward + self.gamma * np.max(self.q_table[self.current_row_index, self.current_column_index]) - old_q_value
        
        new_q_value = old_q_value + self.alpha * temporal_difference
        self.q_table[old_row_index, old_column_index, action_index] = round(new_q_value, 5)
        
    def inf_about_episods(self):
        inf = pygame.font.SysFont('verdana', 15).render('Эпизод № ' + str(self.episod_number), True, (0,0,0))
        screen.blit(inf, (700, 100))
        
    # Метод learning будет отвечать за обучение агента
    def learning(self):
        #if not self.is_terminal_state():
        #выбираем действие, чтобы ходить
        action_index = self.get_next_action()
        while self.check_action(action_index) == False:
            action_index = self.get_next_action()

        old_row_index, old_column_index = self.current_row_index, self.current_column_index         
        self.current_row_index, self.current_column_index = self.get_next_location(action_index)
        
        #self.update_q_table(old_row_index, old_column_index, action_index)
        self.inf_about_episods()


environment_rows = 13
environment_columns = 15


FPS = 5
clock = pygame.time.Clock()
color1 = (228, 206, 242)  # фон
color2 = (94, 53, 242)  # (161, 115, 191)  # стенки
color3 = (29, 30, 38)  # линии
color4 = (242, 5, 135)  # финиш и стартa
color5 =  (140, 92, 242) # ловушки
background_color = color1
scale = 40
(width, height) = (environment_columns * scale + 350 + scale, environment_rows * scale + 50 + scale)
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption('Training')         #заголовок окна
screen.fill(background_color)
pygame.display.flip()



#создаем персонажа игры как объекта класса Q_learning
agent_sonic = Q_learning(environment_rows, environment_columns, screen, scale, True)
agent_sonic.generate_maze()

running = True
while running == True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

            
    screen.fill(background_color) 
    agent_sonic.draw_environment()
    agent_sonic.learning()   
    agent_sonic.draw_agent()
    #agent_sonic.draw_values()
    
    clock.tick(FPS)  
    pygame.display.flip()

pygame.quit()